#FIX ROT 90 LABELS

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

import pathlib
import os
import cv2
import zipfile
from os import path
import shutil
import csv
import random
import numpy as np

Cloning into 'yolov5'...
remote: Enumerating objects: 9893, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 9893 (delta 51), reused 32 (delta 11), pack-reused 9801
Receiving objects: 100% (9893/9893), 10.36 MiB | 21.57 MiB/s, done.
Resolving deltas: 100% (6837/6837), done.
/content/yolov5
     |████████████████████████████████| 596 kB 7.0 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# The next few cells can be used to download and train on LPCV Data Directly
# You must upload all LPCV M4V and CSV files to create the dataset

In [ ]:
%mkdir LPCV
%cd LPCV
%mkdir images
%mkdir labels
%mkdir valid
%mkdir valid/images
%mkdir valid/labels
%mkdir train
%mkdir train/images
%mkdir train/labels
%cd ..

/content/yolov5/LPCV
/content/yolov5


In [37]:
%rm -r LPCV/train
%rm -r LPCV/valid
%mkdir ./LPCV/valid
%mkdir ./LPCV/valid/images
%mkdir ./LPCV/valid/labels
%mkdir ./LPCV/train
%mkdir ./LPCV/train/images
%mkdir ./LPCV/train/labels

In [ ]:
def capture_frames(video, variety):
  vidcap = cv2.VideoCapture(video)
  success,image = vidcap.read()
  count = 0
  while success:
    cv2.imwrite("LPCV/images/{}_frame{}.jpg".format(variety,count), image)     # save frame as JPEG file      
    success,image = vidcap.read()
    count += 1
  print("Finished Reading " + video)

In [ ]:
def create_labels(video_csv, variety):
  with open(video_csv, newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    yolo_labels = []
    for row in csv_reader:
      frame = row[0]
      if row[0] == "Frame":
        continue
      else:
        with open("LPCV/labels/"+variety+"_frame" + str(frame) + ".txt", "a") as yololabelfile:
          yoloarray = row
          del yoloarray[2:3]
          del yoloarray[0:1]
          for item in yoloarray:
            yololabelfile.write("%s " % item)
          yololabelfile.write("\n")

In [ ]:
capture_frames('4p1b_01A2.m4v','4p1b')
capture_frames('5p2b_01A1.m4v','5p2b')
capture_frames('5p4b_01A2.m4v','5p4b')
capture_frames('5p5b_03A1.m4v','5p5b')
capture_frames('7p3b_02M.m4v','7p3b')

Finished Reading 4p1b_01A2.m4v
Finished Reading 5p2b_01A1.m4v
Finished Reading 5p4b_01A2.m4v
Finished Reading 5p5b_03A1.m4v
Finished Reading 7p3b_02M.m4v


In [ ]:
create_labels('4p1b_01A2.csv','4p1b')
create_labels('5p2b_01A1.csv','5p2b')
create_labels('5p4b_01A2.csv','5p4b')
create_labels('5p5b_03A1.csv','5p5b')
create_labels('7p3b_02M.csv','7p3b')

In [ ]:
from tensorflow import keras
import tensorflow_datasets as tfds
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import os
import tensorflow as tf
import numpy as np
import glob
from shutil import copyfile
from PIL import Image

In [ ]:
for root, dirs, files in os.walk('LPCV/labels/', topdown=False):
    for filename in files:
      num = filename[10:filename.find(".")]
      name = filename[:10]
      video_file = name+num+".jpg"
      copyfile('LPCV/labels/'+filename, 'LPCV/train/labels/'+filename)
      copyfile('LPCV/images/'+video_file, 'LPCV/train/images/'+video_file)

In [ ]:
!zip -r LPCVtrain.zip

#To save LPCV image and labels, zip and download the LPCV folder at this point. In the future the folder can be uploaded and the above cells are uneccessary

In [38]:
%rm -r ./LPCV

In [39]:
!unzip LPCVtrain.zip

Archive:  LPCVtrain.zip
   creating: LPCV/train/
   creating: LPCV/train/labels/
  inflating: LPCV/train/labels/5p4b_frame929.txt  
  inflating: LPCV/train/labels/5p5b_frame7.txt  
  inflating: LPCV/train/labels/7p3b_frame8.txt  
  inflating: LPCV/train/labels/5p2b_frame896.txt  
  inflating: LPCV/train/labels/7p3b_frame3678.txt  
  inflating: LPCV/train/labels/5p5b_frame932.txt  
  inflating: LPCV/train/labels/4p1b_frame7.txt  
  inflating: LPCV/train/labels/5p4b_frame0.txt  
  inflating: LPCV/train/labels/4p1b_frame1.txt  
  inflating: LPCV/train/labels/5p2b_frame898.txt  
  inflating: LPCV/train/labels/5p4b_frame2.txt  
  inflating: LPCV/train/labels/4p1b_frame2.txt  
  inflating: LPCV/train/labels/5p5b_frame4.txt  
  inflating: LPCV/train/labels/5p2b_frame897.txt  
  inflating: LPCV/train/labels/5p5b_frame2.txt  
  inflating: LPCV/train/labels/5p2b_frame1.txt  
  inflating: LPCV/train/labels/5p4b_frame935.txt  
  inflating: LPCV/train/labels/5p2b_frame2.txt  
  inflating: LPCV/trai

In [40]:
%mkdir ./LPCV/valid
%mkdir ./LPCV/valid/images
%mkdir ./LPCV/valid/labels

In [41]:
def contrast_image(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.random_contrast(img, 2, 5, 42)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    copy_label(image, "contrast", folder)
def noise_image(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.stateless_random_jpeg_quality(img, 2, 20, (42,42))
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    copy_label(image, "noise", folder)
def grayscale_image(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.rgb_to_grayscale(img)
    img = img.numpy()
    img = img.reshape((img.shape[0], img.shape[1]))
    im = Image.fromarray(img, 'L')
    im.save(folder+'/images/'+new_image)
    copy_label(image, "grayscale", folder)
def saturate_image(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.random_saturation(img, 2, 120, 42)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    copy_label(image, "saturate", folder)
def hue_image(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.random_hue(img, 0.1)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    copy_label(image, "hue", folder)
def flip_image_lr(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.flip_left_right(img)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    flip_label_lr(image[:image.find('.')]+".txt", image[:image.find('.')]+"_flip_lr.txt", folder)
def flip_image_ud(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.flip_up_down(img)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    flip_label_ud(image[:image.find('.')]+".txt", image[:image.find('.')]+"_flip_ud.txt", folder)
def rot_90(image, new_image, folder):
    img = Image.open(folder+'/images/'+image)
    img = np.array(img)
    img = tf.image.rot90(img)
    img = img.numpy()
    im = Image.fromarray(img)
    im.save(folder+'/images/'+new_image)
    label_rot_90(image[:image.find('.')]+".txt", image[:image.find('.')]+"_rot_90.txt", folder)
def flip_label_lr(label_file, new_label_file, folder):
    label_file = folder+"/labels/"+label_file
    new_label_file = folder+"/labels/"+new_label_file
    with open(label_file) as f:
        lines = f.readlines()
        new_lines = []
        for x in lines:
            words = x.split(" ")
            words[1] = str(1-float(words[1]))
            new_lines.append(" ".join(words))
    f.close()
    with open(new_label_file, 'w') as f:
        for x in new_lines:
            f.write(x)
def flip_label_ud(label_file, new_label_file, folder):
    label_file = folder+"/labels/"+label_file
    new_label_file = folder+"/labels/"+new_label_file
    with open(label_file) as f:
        lines = f.readlines()
        new_lines = []
        for x in lines:
            words = x.split(" ")
            words[2] = str(1-float(words[2]))
            new_lines.append(" ".join(words))
    f.close()
    with open(new_label_file, 'w') as f:
        for x in new_lines:
            f.write(x)
def label_rot_90(label_file, new_label_file, folder):
    label_file = folder+"/labels/"+label_file
    new_label_file = folder+"/labels/"+new_label_file
    with open(label_file) as f:
        lines = f.readlines()
        new_lines = []
        for x in lines:
            words = x.split(" ")
            x = words[1]
            y = words[2]
            w = words[3]
            h = words[4][:len(words[4])-1]
            words[1] = y
            words[2] = str(1-float(x))
            words[3] = h
            words[4] = w
            new_lines.append(" ".join(words))
    f.close()
    with open(new_label_file, 'w') as f:
        for x in new_lines:
            f.write(x)
def copy_label(image, augmentation, folder):
    filename = image[:image.find('.')]
    new_filename = folder+'/labels/'+filename+'_'+augmentation+'.txt'
    filename = folder+'/labels/'+filename+'.txt'
    copyfile(filename, new_filename)

In [42]:
aug_list = ['_contrast', '_noise', '_flip_lr', '_flip_ud', '_rot_90', '_saturate', '_hue']
data_dir = "./LPCV/train"

In [43]:
for root, dirs, files in os.walk(data_dir+'/images/', topdown=False):
    for file in files:
      count = 0
      for j in aug_list:
        filename = file[:file.find('.')]+j+file[file.find('.'):]
        if count==0: contrast_image(file, filename, data_dir)
        if count==1: noise_image(file, filename, data_dir)
        if count==2: flip_image_lr(file, filename, data_dir)
        if count==3: flip_image_ud(file, filename, data_dir)
        if count==4: rot_90(file, filename, data_dir)
        if count==5: saturate_image(file, filename, data_dir)
        if count==6: hue_image(file, filename, data_dir)
        count+=1

In [44]:
for root, dirs, files in os.walk(data_dir+'/images/', topdown=False):
    for file in files:
        if file == '.DS_Store':
            continue
        filename = file[:file.find('.')]+"_grayscale"+file[file.find('.'):]
        grayscale_image(file, filename, data_dir)

In [45]:
def moveFiles(src, dst):
  files = []
  for i in os.listdir(src):
    files.append(i)
  files = sorted(files)
  listt = np.random.RandomState(seed=12).permutation(files)[:30]
  for f in listt:
    shutil.copy(os.path.join(src, f), dst)
    p = os.path.join(src, f)
    os.remove(p)

In [46]:
moveFiles("LPCV/train/images", "LPCV/valid/images")
moveFiles("LPCV/train/labels", "LPCV/valid/labels")

In [47]:
data = open("data.yaml", "w")
data.write("train: ./LPCV/train/images\nval: ./LPCV/valid/images\nnc: 2\nnames: [\'person\', \'sports-ball\']")
data.close()

In [ ]:
!python train.py --img 416 --batch 150 --epochs 20 --data ./data.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=./data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=20, batch_size=150, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-86-g7473f0f torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scal

In [ ]:
 !python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.05 --source 5p5b_03A1.m4v

In [ ]:
!python val.py --weights runs/train/exp4/weights/best.pt --img 640 --data data.yaml

val: data=data.yaml, weights=['runs/train/exp4/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.0-86-g7473f0f torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning 'LPCV/valid/labels.cache' images and labels... 30 found, 0 missing, 0 empty, 0 corrupted: 100% 30/30 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 1/1 [00:06<00:00,  6.97s/it]
                 all         30        241      0.974      0.883      0.928      0.589
              person         30        147      0.989          1      0.995      0.734
         sports-ball         30         94       0.96      0.766     

In [ ]:
print("hi")

hi
